## 上週我們學了
1. Requests用法
2. BeautifulSoup提取HTML Tag
3. 爬蟲設計策略

In [2]:
from __future__ import print_function

In [27]:
import requests
from bs4 import BeautifulSoup
import json

ptt_base_url = "https://www.ptt.cc"
base_url = 'https://www.ptt.cc/bbs/Food/index{}.html'

"""爬取總頁數"""
base_page_raw = requests.get(base_url.format(''))
base_page = BeautifulSoup(base_page_raw.text, 'html5lib')
total_page = int(base_page\
             .select('a.btn.wide')[1]['href']\
             .split('index')[1]\
             .split('.')[0]) + 1


for page in xrange(total_page-1, total_page-2, -1): # xrange(起始值,結束值,step)
    target = base_url.format(page)
    #print(target)
    current_page_raw = requests.get(target)
    current_page = BeautifulSoup(current_page_raw.text, 'html5lib')
    
    articles = current_page.select('div.r-ent') # List of Tags


    for article in articles:
        try:
            article_dict = {}
            #title = article.select('.title')[0].text.strip()
            #author = article.select('.meta')[0].select('.author')[0].text.strip()
            #date = article.select('.meta')[0].select('.date')[0].text.strip()
            article_link = ptt_base_url + article.select('.title')[0].select('a')[0]['href']
            print(article_link)
            print("========================")
            
            article_raw = requests.get(article_link)
            article = BeautifulSoup(article_raw.text, 'html5lib')
            article_meta_value = article.select(".article-meta-value")
            #print(article_meta_value)
            article_dict['title'] = article_meta_value[0].text
            article_dict['author'] = article_meta_value[2].text
            article_dict['date'] = article_meta_value[3].text
            #print(title, author, date)
            
            article_dict['pushes'] = []
            pushes = article.select('.push')
            for push in pushes:
                push_dict = {} # push_tag, push_userid, push_content, push_date
                #print(push_first)
                push_dict['push_tag'] = push.select('.push-tag')[0].text
                push_dict['push_userid'] = push.select('.push-userid')[0].text
                push_dict['push_content'] = push.select('.push-content')[0].text
                push_dict['push_date'] = push.select('.push-ipdatetime')[0].text
                article_dict['pushes'].append(push_dict)
                #print(push)
            
            main_content = article.select('#main-content')[0]
            [s.extract() for s in main_content('div')]
            article_dict['content'] = main_content.text.split(u'※ ')[0]
            article_json = json.dumps(article_dict, ensure_ascii=False).encode('utf-8')
            with open('./PTT/ptt_{}'.format(article_dict['date']), 'w') as f:
                f.write(article_json)
            print(article_dict)
            
            
        except IndexError as e:
            print("Article Deleted")

https://www.ptt.cc/bbs/Food/M.1466603980.A.8A6.html
{'date': u'Wed Jun 22 21:59:37 2016', 'content': u'\n\n\u9910\u5ef3\u540d\u7a31\uff1aCHANTEZ P\xe2tisserie & Salon de th\xe9 \u7a7f\u77f3\u751c\u9ede\u7a7a\u9593\n\u6d88\u8cbb\u6642\u9593\uff1a2016\u5e74/6\u6708\n\u96fb\u8a71\uff1a02-27786565\n\u5730\u5740\uff1a\u53f0\u5317\u5e02\u5927\u5b89\u5340\u6fdf\u5357\u8def\u4e09\u6bb531\u865f\n\u71df\u696d\u6642\u9593\uff1a12:00~21:00 (\u4f11\u5047\u65e5\u65bcFB\u516c\u544a)\n\nBlog\u5716\u6587\u7248\uff1ahttp://morrisyu.com/archives/4014\n\n\u539f\u672c\u4f4d\u5728\u6c11\u751f\u793e\u5340\u7684Project 25 \u751cx\u98df\u8a08\u5283\uff0c\u662f\u4e00\u5bb6\u751c\u9ede\u5de5\u4f5c\u5ba4\uff0c\u66fe\u7d93\u9020\u8a2a\u904e\u4e00\u6b21\uff0c\u53ea\u8a18\n\u5f97\u5e97\u9762\u5c0f\u5c0f\u4e00\u500b\uff0c\u5167\u7528\u5e7e\u5e7e\u500b\u4f4d\u7f6e\uff0c\u662f\u4ee5\u9810\u5b9a\u5916\u5e36\u5c45\u591a\u7684\u751c\u9ede\u5e97\uff0c\u6c92\u60f3\u5230\u73fe\u5728\u64f4\u5927\u71df\u696d\uff0c\n\u5728\u637

In [45]:
mystr = "寶寶" # this is a string
print(type(mystr))
print(repr(mystr)) # Python看到的是這個
print('=============我們decode這個str，變成unicode================')
mystr_decoded = mystr.decode('utf-8')
print(repr(mystr_decoded))
print(type(mystr_decoded))
print('============我們encode剛才decode的unicode回去str=================')
mystr_re_encoded = mystr_decoded.encode('utf-8')
print(repr(mystr_re_encoded))
print(type(mystr_re_encoded))

<type 'str'>
'\xe5\xaf\xb6\xe5\xaf\xb6'
=============我們decode這個str，變成unicode================
u'\u5bf6\u5bf6'
<type 'unicode'>
============我們encode剛才decode的unicode回去str=================
'\xe5\xaf\xb6\xe5\xaf\xb6'
<type 'str'>


## 政府採購網

In [35]:
url = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance"
post_date = {
    "method":"search",
    "searchMethod":"true",
    "searchTarget":"ATM",
    "orgName":"",
    "orgId":"",
    "hid_1":"1",
    "tenderName":"",
    "tenderId":"",
    "tenderStatus":"4,5,21,29",
    "tenderWay":"",
    "awardAnnounceStartDate":"105/06/22",
    "awardAnnounceEndDate":"105/06/22",
    "proctrgCate":"",
    "tenderRange":"",
    "minBudget":"",
    "maxBudget":"",
    "item":"",
    "hid_2":1,
    "gottenVendorName":"",
    "gottenVendorId":"",
    "hid_3":1,
    "submitVendorName":"",
    "submitVendorId":"",
    "location":"",
    "priorityCate":"",
    "isReConstruct":"",
    "btnQuery":"查詢"
}


rs = requests.session() # 取得新的session
res = rs.post(url, data=post_date)
#print(res.text)
res2 = rs.get("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=2")
print(res2.text)

# http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance
# http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=2










<link rel="stylesheet" href="../styles/base.css" type="text/css">
<link rel="stylesheet" href="/styles/text.css" type="text/css">
<link rel="stylesheet" href="/styles/css.css" type="text/css">
<link href="../styles/confirm.css" rel="stylesheet" type="text/css" />
<style type="text/css">
a:visited {
	text-decoration: none;
	color: #444444;
}

a:hover {
	text-decoration: none;
	color: #0066CC;
}

a:link {
	text-decoration: none;
	color: #444444;
}
</style>

	
	
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<meta http-equiv="Pragma" content="no-cache">
<meta http-equiv="Cache-Control" content="no-cache">
<meta http-equiv="Expires" content="0">
<title>政府電子採購網</title>
<link href="/pis/styles/css_m.css" rel="stylesheet" type="text/css">
<script type="text/javascript" src="/pis/scripts/jquery-1.3.2.js"></script>
<script type="text/javascript" src=

## 正規表達式(Regular Expression)

In [36]:
import re

In [43]:
# match = re.search(pattern, str)
mystr = "an example word:do4!!!!!"
match = re.search(r'word:...', mystr)
print(match.group())

word:do4


## 基本Pattern
1. a-z, A-Z, 0-9 -- 一對一match
2. .(句點) -- 任意字元，除\n
3. \w -- [a-zA-Z0-9_]
4. \s -- [   \t\n\r\f] 空白符號
5. \d -- [0-9]
6. ^ -- 開頭
7. $ -- 結尾


In [57]:
mypet = 'do123876567876567667hjkhkj'
match = re.search(r'zzz', mypet)
match = re.search(r'...', mypet)
match = re.search(r'\w\w', mypet)
match = re.search(r'\d\d\d', mypet)
match = re.search(r'^\d\d\d', mypet)
match = re.search(r'\d\d\d$', mypet)



print(match.group())



123


## 重複(repetition)
1. +, 1到n次
2. *, 0到n次
3. ?, 0或1次

In [79]:
mypet = 'ddddddgddd'

#match = re.search(r'o+', mypet)
match = re.search(r'^d*', mypet)
match = re.search(r'(?P<mygrp>.)', mypet)
print(match.group('mygrp'))

d


In [76]:
m = re.match(r"(?P<FirstName>\w+) (?P<LastName>\w+)", "David Chiu")
m.group("FirstName")

'David'

## 以EMail為例

In [81]:
somestr = 'yello green taiwan ianchen06@gmail.com nice Ilovepython'
match = re.search(r'\w+@\w+', somestr)
print(match.group(0))


ianchen06@gmail


- [    ] -> OR
- {} -> number of times

In [83]:
somestr = 'yello green taiwan ian.chen06@gmail.com nice Ilovepython'
match = re.search(r'[\w.]+@[\w.]+', somestr)
print(match.group(0))

ian.chen06@gmail.com


## Group Extraction

In [93]:
somestr = 'yello green taiwan ian.chen06@gmail.com nice Ilovepython'
match = re.search(r'([\w.]+)@([\w.]+)', somestr)
print(match.group(0))
print(match.group(1))
print(match.group(2))



ian.chen06@gmail.com
ian.chen06
gmail.com


## Python only feature
1. (?P&lt;name&gt;xxxxxxx) -> named group

In [95]:
somestr = 'yello green taiwan ian.chen06@gmail.com nice Ilovepython'
match = re.search(r'(?P<username>[\w.]+)@(?P<domain>[\w.]+)', somestr)
print(match.group('username'))
print(match.group('domain'))

ian.chen06
gmail.com


## 金錢轉換

In [100]:
import re
a = '他欠了我25,000元'
b = '我欠了他250,000,000元'
m1 = re.search('25,000', a)
match.group(0)

'25,000'

In [105]:
m = re.search(r'[023456789,]+', b)
print(m.group(0))
print(m.group(0).split(','))
print("".join(m.group(0).split(',')))

250,000,000
['250', '000', '000']
250000000


In [107]:
m = re.search(r'[023456789,]+', b)
print(m.group(0))
res = re.sub(r',', "", m.group(0)) # re.sub(pattern, replacement, str)
print(res)

250,000,000
250000000


In [115]:
phones = ['我的電話是 0912345678 Call Me', '0922-222-222 我餓了', \
          '等妳呦 0911-222333', '真心不騙 09123482384060192381']

for phone in phones:
    m = re.search(r'09\d+', phone)
    print(m.group(0))
print("=====================")
for phone in phones:
    m = re.search(r'09\d{2}-?\d{3}-?\d{3}', phone) # r'09\d\d'
    print(m.group(0))

0912345678
0922
0911
09123482384060192381
0912345678
0922-222-222
0911-222333
0912348238


## 空白處理

In [128]:
a = '   1!23!      '
a_no_whitespace = a.strip()
print(a.strip())
print(a.lstrip())
print(a.rstrip())
print(a_no_whitespace.strip('!'))

b = '   !你好!'
print(b.strip('!'))

1!23!
1!23!      
   1!23!
1!23
   !你好
